In [1]:
import pandas as pd
import numpy as np
from propagate import propagate
import rebound
from skyfield.api import Topos, Loader
from scipy.optimize import newton

In [2]:
#Load JPL ephemeris
load = Loader('./Skyfield-Data', expire=False)
planets = load('de423.bsp')
#Load minor planets orbits
orbits = pd.read_csv('MPCORB.csv')

/Users/hsingwel/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#Use Cerers (as the represented of main-belt Asteroids) and Phaethon (near-Earth Asteroid) for testing
match_mps = [1, 3200]
objects = orbits[orbits.name.isin(match_mps)]

In [4]:
#make sure we get the right objects
objects

,H,M,W,a,e,epoch,i,name,w
0,3.3,1.350399,1.401596,2.769165,0.076009,2458600.5,0.184901,1,1.284521
3199,14.6,5.479269,4.628896,1.271373,0.889898,2458600.5,0.388516,3200,5.623198


In [5]:
#Calcuate their barycentric Cartesian state vectors
p = propagate(np.array(objects.a), np.array(objects.e), np.array(objects.i), np.array(objects.w), 
              np.array(objects.W), np.array(objects.M), np.array(objects.epoch), np.array(objects.epoch), helio=True)

In [6]:
#check the positions
print(p.X, p.Y, p.Z, p.VX, p.VY, p.VZ)

[-1.35826654  0.41601615] [-2.36506867  1.34153003] [0.17534347 0.12481611] [ 0.00845672 -0.00917263] [-0.00587557 -0.00966178] [-0.00174554 -0.00340801]


In [7]:
#prepare for n_body simulation, here we use 'rebound' integrator 
sim = rebound.Simulation()
sim.units = ('day', 'AU', 'Msun')
sim.integrator = "IAS15"#"mercurius"

In [8]:
 def equa_to_ecl(X0,Y0,Z0):
    epsilon =  23.43929111 * np.pi/180.
    X = X0
    Y = Y0 * np.cos(epsilon) + Z0 * np.sin(epsilon)
    Z = -Y0 * np.sin(epsilon) + Z0 * np.cos(epsilon)
    return X, Y, Z

In [9]:
#here we load the Sun and planetary positions
epoch0 = float(objects.epoch[0])
ts = load.timescale()
t = ts.tdb(jd=epoch0)
Sun = planets['Sun']
Mercury = planets[1]
Venus = planets[2]
Earth = planets[3]
Mars = planets[4]
Jupiter = planets[5]
Saturn = planets[6]
Uranus = planets[7]
Neptune = planets[8]
Sun_x, Sun_y, Sun_z = Sun.at(t).position.au
Sun_vx, Sun_vy, Sun_vz = Sun.at(t).velocity.au_per_d
Sun_x, Sun_y, Sun_z = equa_to_ecl(Sun_x, Sun_y, Sun_z)
Sun_vx, Sun_vy, Sun_vz = equa_to_ecl(Sun_vx, Sun_vy, Sun_vz)
Sun_mass = 1

Mercury_x, Mercury_y, Mercury_z = Mercury.at(t).position.au
Mercury_vx, Mercury_vy, Mercury_vz = Mercury.at(t).velocity.au_per_d
Mercury_x, Mercury_y, Mercury_z = equa_to_ecl(Mercury_x, Mercury_y, Mercury_z)
Mercury_vx, Mercury_vy, Mercury_vz = equa_to_ecl(Mercury_vx, Mercury_vy, Mercury_vz)
Mercury_mass = 1.6601367952719304E-07

Venus_x, Venus_y, Venus_z = Venus.at(t).position.au
Venus_vx, Venus_vy, Venus_vz = Venus.at(t).velocity.au_per_d
Venus_x, Venus_y, Venus_z = equa_to_ecl(Venus_x, Venus_y, Venus_z)
Venus_vx, Venus_vy, Venus_vz = equa_to_ecl(Venus_vx, Venus_vy, Venus_vz)
Venus_mass = 2.4478383396645447E-06

Earth_x, Earth_y, Earth_z = Earth.at(t).position.au
Earth_vx, Earth_vy, Earth_vz = Earth.at(t).velocity.au_per_d
Earth_x, Earth_y, Earth_z = equa_to_ecl(Earth_x, Earth_y, Earth_z)
Earth_vx, Earth_vy, Earth_vz = equa_to_ecl(Earth_vx, Earth_vy, Earth_vz)
Earth_mass = 3.0404326462685257E-06

Mars_x, Mars_y, Mars_z = Mars.at(t).position.au
Mars_vx, Mars_vy, Mars_vz = Mars.at(t).velocity.au_per_d
Mars_x, Mars_y, Mars_z = equa_to_ecl(Mars_x, Mars_y, Mars_z)
Mars_vx, Mars_vy, Mars_vz = equa_to_ecl(Mars_vx, Mars_vy, Mars_vz)
Mars_mass = 3.2271514450538743E-07

Jupiter_x, Jupiter_y, Jupiter_z = Jupiter.at(t).position.au
Jupiter_vx, Jupiter_vy, Jupiter_vz = Jupiter.at(t).velocity.au_per_d
Jupiter_x, Jupiter_y, Jupiter_z = equa_to_ecl(Jupiter_x, Jupiter_y, Jupiter_z)
Jupiter_vx, Jupiter_vy, Jupiter_vz = equa_to_ecl(Jupiter_vx, Jupiter_vy, Jupiter_vz)
Jupiter_mass = 9.547919384243222E-04

Saturn_x, Saturn_y, Saturn_z = Saturn.at(t).position.au
Saturn_vx, Saturn_vy, Saturn_vz = Saturn.at(t).velocity.au_per_d
Saturn_x, Saturn_y, Saturn_z = equa_to_ecl(Saturn_x, Saturn_y, Saturn_z)
Saturn_vx, Saturn_vy, Saturn_vz = equa_to_ecl(Saturn_vx, Saturn_vy, Saturn_vz)
Saturn_mass = 2.858859806661029E-04

Uranus_x, Uranus_y, Uranus_z = Uranus.at(t).position.au
Uranus_vx, Uranus_vy, Uranus_vz = Uranus.at(t).velocity.au_per_d
Uranus_x, Uranus_y, Uranus_z = equa_to_ecl(Uranus_x, Uranus_y, Uranus_z)
Uranus_vx, Uranus_vy, Uranus_vz = equa_to_ecl(Uranus_vx, Uranus_vy, Uranus_vz)
Uranus_mass = 4.3662440433515637E-05

Neptune_x, Neptune_y, Neptune_z = Neptune.at(t).position.au
Neptune_vx, Neptune_vy, Neptune_vz = Neptune.at(t).velocity.au_per_d
Neptune_x, Neptune_y, Neptune_z = equa_to_ecl(Neptune_x, Neptune_y, Neptune_z)
Neptune_vx, Neptune_vy, Neptune_vz = equa_to_ecl(Neptune_vx, Neptune_vy, Neptune_vz)
Neptune_mass = 5.151389020535497E-05

In [10]:
#Add them into simulation
sim.add(m=Sun_mass, x=Sun_x, y=Sun_y, z=Sun_z, vx=Sun_vx, vy=Sun_vy, vz=Sun_vz)
sim.add(m=Mercury_mass, x=Mercury_x, y=Mercury_y, z=Mercury_z, vx=Mercury_vx, vy=Mercury_vy, vz=Mercury_vz)
sim.add(m=Venus_mass, x=Venus_x, y=Venus_y, z=Venus_z, vx=Venus_vx, vy=Venus_vy, vz=Venus_vz)
sim.add(m=Earth_mass, x=Earth_x, y=Earth_y, z=Earth_z, vx=Earth_vx, vy=Earth_vy, vz=Earth_vz)
sim.add(m=Mars_mass, x=Mars_x, y=Mars_y, z=Mars_z, vx=Mars_vx, vy=Mars_vy, vz=Mars_vz)
sim.add(m=Jupiter_mass, x=Jupiter_x, y=Jupiter_y, z=Jupiter_z, vx=Jupiter_vx, vy=Jupiter_vy, vz=Jupiter_vz)
sim.add(m=Saturn_mass, x=Saturn_x, y=Saturn_y, z=Saturn_z, vx=Saturn_vx, vy=Saturn_vy, vz=Saturn_vz)
sim.add(m=Uranus_mass, x=Uranus_x, y=Uranus_y, z=Uranus_z, vx=Uranus_vx, vy=Uranus_vy, vz=Uranus_vz)
sim.add(m=Neptune_mass, x=Neptune_x, y=Neptune_y, z=Neptune_z, vx=Neptune_vx, vy=Neptune_vy, vz=Neptune_vz)

In [11]:
#Add minor palnets into sumulation
for i in range(len(p.X)):
    X, Y, Z = p.X[i],p.Y[i], p.Z[i]
    VX, VY, VZ = p.VX[i],p.VY[i], p.VZ[i]
    sim.add(m=0, x=X, y=Y, z=Z, vx=VX, vy=VY, vz=VZ)

In [12]:
#Check the simulation status before intergation start
sim.status()

---------------------------------
REBOUND version:     	3.6.2
REBOUND built on:    	Jun  7 2018 14:27:59
Number of particles: 	11
Selected integrator: 	ias15
Simulation time:     	0.0000000000000000e+00
Current timestep:    	0.001000
---------------------------------
<rebound.Particle object, m=1.0 x=-0.0016997040830256903 y=0.007591942332089701 z=-3.294690197843059e-05 vx=-8.347175743351958e-06 vy=7.503819207352465e-07 vz=2.164050466367272e-07>
<rebound.Particle object, m=1.6601367952719304e-07 x=0.2490276285473541 y=-0.3367327496768238 z=-0.051169932867923804 vx=0.017127924764256945 vy=0.01792987746920543 vz=-0.00010675872004990532>
<rebound.Particle object, m=2.4478383396645447e-06 x=0.5980231225085578 y=-0.4032277742846196 z=-0.04027844023235563 vx=0.011298421958249658 vy=0.01660693268110823 vz=-0.00042441132947431716>
<rebound.Particle object, m=3.0404326462685257e-06 x=-0.8137654121208079 y=-0.5867872155528668 z=-3.657956725799938e-06 vx=0.009872927928323776 vy=-0.013947017175980

In [13]:
#Let's start the intergation, here we intergate to 3000 days before the current epoch
sim.integrate(-3000, exact_finish_time=1) 

In [14]:
#Check the status after intergation
sim.status()

---------------------------------
REBOUND version:     	3.6.2
REBOUND built on:    	Jun  7 2018 14:27:59
Number of particles: 	11
Selected integrator: 	ias15
Simulation time:     	-3.0000000000000000e+03
Current timestep:    	-1.663035
---------------------------------
<rebound.Particle object, m=1.0 x=-0.0041362149015444155 y=0.00047756304908454363 z=1.564283191402366e-05 vx=1.1507755023897295e-06 vy=-6.071880038712498e-06 vz=-1.347542593637939e-08>
<rebound.Particle object, m=1.6601367952719304e-07 x=0.0637799783063576 y=-0.45100370231777404 z=-0.043104267701962254 vx=0.022180239852419386 vy=0.005623501795565629 vz=-0.0015751171311462135>
<rebound.Particle object, m=2.4478383396645447e-06 x=-0.6805228794541119 y=-0.24813877936446785 z=0.035645846856020645 vx=0.006843031541450925 vy=-0.019082700632164846 vz=-0.0006562233848094066>
<rebound.Particle object, m=3.0404326462685257e-06 x=-0.74521355163232 y=0.6513719009116126 z=-3.6939646613148885e-08 vx=-0.011632002516959028 vy=-0.0129977

In [15]:
#now the postions of Planets and minor planets are changed to 3000 days before
#take the postions of our asteroids
ceres= sim.particles[-2]
Phaethon = sim.particles[-1]

In [16]:
#to calculate the current orbital elements we define a function to transfer state vector to keplerian orbit
def xyz_to_kep(X, Y, Z, VX, VY, VZ, u):
    # compute the barycentric distance r
    r = (X**2 + Y**2 + Z**2)**0.5
    rrdot = (X*VX + Y*VY + Z*VZ)
    # compute the specific angular momentum h
    hx = Y * VZ - Z * VY
    hy = Z * VX - X * VZ
    hz = X * VY - Y * VX
    h = (hx**2 + hy**2 + hz**2)**0.5
    # compute eccentricity vector
    ex = (VY * hz - VZ * hy)/u - X/r
    ey = (VZ * hx - VX * hz)/u - Y/r
    ez = (VX * hy - VY * hx)/u - Z/r
    e = (ex**2+ey**2+ez**2)**0.5
    # compute vector n
    nx = -hy
    ny = hx
    nz = 0
    n = (nx**2 + ny**2)**0.5
    # compute true anomaly v, the angle between e and r
    v = np.arccos((ex * X + ey * Y + ez * Z) / (e*r))
    v[rrdot<0] = 2*np.pi - v[rrdot<0]
    # compute inclination
    i = np.arccos(hz/h)
    # compute eccentric anomaly E
    E = 2*np.arctan2((1-e)**0.5*np.sin(v/2.), (1+e)**0.5*np.cos(v/2.))
    # compute ascending node
    node = np.arccos(nx/n)
    node[ny<0] = 2*np.pi - node[ny<0]
    # compute argument of periapsis, the angle between e and n
    arg = np.arccos((nx * ex + ny * ey + nz *ez) / (n*e))
    arg[ez<0] = 2*np.pi - arg[ez<0]
    # compute mean anomaly
    M = E - e * np.sin(E)
    M[M<0] += 2*np.pi
    # compute a
    a = 1/(2/r - (VX**2+VY**2+VZ**2)/u)
    return a, e, i, arg, node, M   

In [17]:
u_bary = 2.9630927492415936E-04 # standard gravitational parameter, GM. M is the mass of sun + all planets 
x, y, z = np.array([ceres.x]), np.array([ceres.y]), np.array([ceres.z])
vx, vy, vz = np.array([ceres.vx]), np.array([ceres.vy]), np.array([ceres.vz])
a, e, i, arg, node, M = xyz_to_kep(x, y, z, vx, vy, vz, u_bary)

In [18]:
#now calcuate the observer quantities ra and dec
p = propagate(a, e, i, arg, node, M, epoch0-3000, epoch0-3000, helio = False)
print(p.ra*180/np.pi, p.dec*180/np.pi, epoch0-3000)

[318.1933104] [-22.69225459] 2455600.5


In [19]:
#Do it again for Phaethon
x, y, z = np.array([Phaethon.x]), np.array([Phaethon.y]), np.array([Phaethon.z])
vx, vy, vz = np.array([Phaethon.vx]), np.array([Phaethon.vy]), np.array([Phaethon.vz])
a, e, i, arg, node, M = xyz_to_kep(x, y, z, vx, vy, vz, u_bary)

In [20]:
p = propagate(a, e, i, arg, node, M, epoch0-3000, epoch0-3000, helio = False)
print(p.ra*180/np.pi, p.dec*180/np.pi, epoch0-3000)

[354.03307187] [12.44211394] 2455600.5
